In [1]:
from sympy import *
import numpy as np
import math
import functools
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
class TreeVar(object):
    def __init__(self, level):
        self.level = level
    def __repr__(self):
        return 'x_%d' % self.level
    def __next__(self):
        return TreeVar(self.level+1)

In [7]:
x = TreeVar(0)

In [8]:
x

x_0

In [9]:
next(x)

x_1

In [24]:
class TreeExp(object):
    def __init__(self):
        self.terms = []
        
    def __add__(self, other):
        result = TreeExp()
        if isinstance(other, TreeExp):
            result.terms = self.terms + other.terms
        elif isinstance(other, TreeTerm):
            result.terms = self.terms + [other]
        return result
    
    def __repr__(self):
        if not self.terms:
            return '0'
        return ' + '.join([t.__repr__() for t in self.terms])

In [28]:
class TreeTerm(object):
    def __init__(self):
        self.factors = []
        
    def __mul__(self, other):
        result = TreeTerm()
        if isinstance(other, TreeTerm):
            result.factors = self.factors + other.factors
        elif isinstance(other, TreeFactor):
            result.factors = self.factors + [other]
    
        return result
    
    def __repr__(self):
        if not self.factors:
            return '1'
        return '*'.join([f.__repr__() for f in self.factors])

In [29]:
class TreeFactor:
    def __init__(self, var_or_exp):
        self.factor = var_or_exp
    def __repr__(self):
        result = self.factor.__repr__()
        if isinstance(self.factor, TreeExp):
            result = '(' + result + ')'
        return result
    def __mul__(self, other):
        x = TreeTerm()*self
        return x*other

# Grammar
The grammar is:

`TreeExp -> TreeTerm [+ TreeExp]`

`TreeTerm -> TreeFactor [* TreeTerm]`

`TreeFactor -> TreeVar | (TreeExp)`

In [38]:
yy = TreeExp() + TreeFactor(x)*TreeFactor(x)

In [39]:
type(yy)

__main__.TreeExp

In [40]:
yy + TreeFactor(x)

0

In [41]:
init_printing()

In [42]:
x = symbols('x0:9')

In [43]:
x[0]

In [45]:
x[0]*x[0]

Ok clearly I'm a bit rusty on how to program parsers, etc. Anyway, the point here is to expand a little bit of stuff in analytic combinatorics. Basically, we want to $q$-binomialize it, i.e. add a variable. For example, one can model multisets and sets by considering $G_n(x; z) = (1+x_1z)(1+x_2z)\cdots(1+x_nz)$ which give all the subsets of size $k$ in the term $[z^k]G_n(x; z)$ (the elements of thes ubsets are just given by the indices of the $x$-variables that appear). If $1$ is substituted for each $x_i$, then $[z^k]G_n(1; z) = {n \choose k }$. Similarly, multisets are realized as $F_n(x; z) = \frac{1}{(1-x_1z)(1-x_2z) \cdots (1-x_nz)}$, and $[z^k]F_n(1; z) = {n+k-1 \choose k-1}$.

In [46]:
z = symbols('z')

In [49]:
(1/(1-z)**3).series(n=5)

Analytic combinatorics, at least as seen so far, does the substitution $x_i = 1$. This is obviously useful for counting: to count all the terms of size $k$, substitute all $x_i = 1$ in $[z^k] G(1;z)$ for any generating function $G$ that has multiple variables. The construction of Cartesian product is even more interesting, though one should take noncommuting variables to realize this. If $x_i$ are $n$ noncommuting variables, and $z$ is allowed to commute with all of then, then defining $G_n(x;z) = (x_1 z + x_2 z + \cdots + x_n z)^2$ is gives the Cartesian product of the set ${1,\dots, n}$ with itself. All finite sequences can be given by $$
\sum_{n=0}^\infty (x_1 z + x_2 z + \cdots + x_n z)^k.
$$

But much more interesting is the generating function for internal nodes binary trees. As done in analytic combinatorics, a tree can be realied as either empty (or one external node), or a tree, a root (internal node), and another tree. This is expressed in the symbolic method as $T(z) = 1 + zT(z)^2$, for which we can solve for $T(z)$ by the quadratic formula:

$$
T(z) = \frac{1 - \sqrt{1- 4z}}{2z}
$$

for which the coefficients are the Catalan numbers. How do we generate one which gives an actual term in variables? This is pretty tough and interesting. We allow infinitely many noncommuting variables $x_0, x_1, x_2, \dots$. The subscripts correspond to levels of the tree. Define $I(x_i) = x_{i+1}$, the transformation that _increments_ the index of the variable. Allow $z$ to commute with everything, as before. Extend $I$ to be a ring homomorphism, distributing over products and sums, and fixing $z$ and all its powers. Then the equation on a tree with explicit nodes will be:

$$
T(x; z) = 1 + I(T(x;z)) x_0 z I(T(x;z)).
$$

or just, for short:

$$
T = 1 + IT x_0 z IT.
$$

In order to check how correct this is, expand it a couple of times and use the distributive law:

$$
T = 1 + I(1 + IT x_0 z IT) x_0 z I(1+ IT x_0 z IT)
$$
$$
T = 1 + (1 + I^2 T x_1 z I^2 T) x_0 z (1+ I^2T x_1 z I^2T)
$$
$$
T = 1 + x_0 z + (I^2 T x_1 z I^2 T) x_0 z +  x_0 z (I^2T x_1 z I^2T) + ( I^2 T x_1 z I^2 T) x_0 z (I^2T x_1 z I^2T)
$$
$$
T = 1 + x_0 z + (I^2 (1 + IT x_0 z IT) x_1 z I^2 (1 + IT x_0 z IT)) x_0 z +  x_0 z (I^2(1 + IT x_0 z IT) x_1 z I^2(1 + IT x_0 z IT)) + ( I^2 (1 + IT x_0 z IT) x_1 z I^2 (1 + IT x_0 z IT)) x_0 z (I^2(1 + IT x_0 z IT) x_1 z I^2(1 + IT x_0 z IT))
$$
$$
T = 1 + x_0 z + (1 + I^3 T x_2 z I^3T) x_1 z (1 + I^3T x_2 z I^3T) x_0 z +  x_0 z (1 + I^3T x_2 z I^3T) x_1 z (1 + I^3T x_2 z I^3T) + ( (1 + I^3T x_2 z I^3T) x_1 z (1 + I^3T x_2 z I^3T)) x_0 z ((1 + I^3T x_2 z I^3T) x_1 z (1 + I^3T x_2 z I^3T))
$$
$$
T = 1 + x_0 z + (x_1x_0 + x_0 x_1)z^2 + (x_2 x_1 x_0 + x_1 x_2 x_0 + x_0 x_2 x_1 + x_0 x_1 x_2 + x_1 x_0 x_1) z^3 + O(z^4)
$$

Basically, in every generation we get a term that looks like $1 + I^k T x_{k-1} z I^k T$, for which we can choose the $1$ and if not, then the term chosen is going to be $x_{k-1} z$ when the choosing the $1$ inside. You choose $1$ whenever you're maxing out on what exponents. The noncommutativity ensures that all those terms cannot collapse, and one gets $c_k$ distinct terms. If all variables are substituted with the value of $1$, then the coefficient is, as expected, just $c_k$. But if commutativity is allowed without substituting, what do we get? If we let $\ell$ be the number of distinct indices, then the coefficients count the number of $\ell$-level trees consisting of $k$ nodes. For example, there are no $1$-level trees consisting of $2$ nodes, as well as no $3$-or-greater level trees.


Anyway, let's derive the explicit expression for the Catalan numbers, and some asymptotic , since these are central to the development of trees, and lots of developments recur, so it's useful. So we have derived

$$
T(z) = \frac{1-\sqrt{1-4z}}{2z}.
$$

Now as a power series, this is the binomial series with the constant term canceled and a power of $z$ missing:

$$
T(z) = \sum_{n\geq 0} c_n z^n = -\frac{1}{2}\sum_{n \geq 1}{\frac{1}{2} \choose n} (-4)^n z^{n-1} = -\frac{1}{2}\sum_{n \geq 0} {\frac{1}{2} \choose n+1} (-4)^{n+1} z^n
$$

and the expression

$$
{\frac{1}{2} \choose n} = \frac{\left(\frac12\right)\left(-\frac12\right)\left(-\frac{3}{2}\right) \cdots \left(\frac12 - n+1\right)}{n!} = \frac{\left(\frac12\right)\left(-\frac12\right)\left(-\frac{3}{2}\right) \cdots \left(\frac{3-2n}{2}\right)}{n!} = (-1)^{n-1}\frac{1\cdot3\cdot5\cdots(2n-3)}{2^n n!} = (-1)^{n-1}\frac{(2n-2)!}{2^{2n-1} (n!)(n-1)!}  
$$

In that last line we simply filled in all the even numbers. But the product of even numbers up to $2n$ is just the factorial, times $2^n$ which accounts for every extra factor of $2$. Therefore

$$
-\frac{1}{2}{\frac{1}{2} \choose n + 1} =  \frac{(-1)^{n+1}}{2}\frac{(2n)!}{2^{2n+1} (n!)^2 (n+1)} = \frac{(-1)^{n+1}}{ 4^{n+1} (n+1)} {2n \choose n} = (-1)^{n-1}\frac{1\cdot3\cdot5\cdots(2n-3)}{2^n n!} = (-1)^{n-1}\frac{(2n-2)!}{2^{2n-1} (n!)(n-1)!}  .
$$

Equating coefficients, the $(-4)^{n+1}$ cancels out, giving the coefficient

$$
c_n =\frac{1}{n+1} {2n \choose n}.
$$


## This notebook continues as a Sage notebook

In [3]:
z = symbols('z')

In [5]:
T = (1 - sqrt(1-4*z))/(2*z)

In [6]:
T.series(z, n=10)

1 + z + 2*z**2 + 5*z**3 + 14*z**4 + 42*z**5 + 132*z**6 + 429*z**7 + 1430*z**8 + 4862*z**9 + O(z**10)

In [11]:
(z*(T - 1)**2/(1-2*z*T)).series(z, n=13)

z**3 + 6*z**4 + 28*z**5 + 120*z**6 + 495*z**7 + 2002*z**8 + 8008*z**9 + 31824*z**10 + 125970*z**11 + 497420*z**12 + O(z**13)